# Simularium Conversion Tutorial : Custom Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio import Converter

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium custom data Converter consumes spatiotemporal data from any source. It requires the following data:
* **box_size** : *np.ndarray (shape = [3])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
* **times** : *np.ndarray (shape = [timesteps])*
    * A numpy ndarray containing the elapsed simulated time at each timestep
* **n_agents** : *np.ndarray (shape = [timesteps])*
    * A numpy ndarray containing the number of agents that exist at each timestep
* **viz_types** : *np.ndarray (shape = [timesteps, agents])*
    * A numpy ndarray containing the viz type for each agent at each timestep. Current options:
        * 1000 : default,
        * 1001 : fiber (which will require subpoints)
* **positions** : *np.ndarray (shape = [timesteps, agents, 3])*
    * A numpy ndarray containing the XYZ position for each agent at each timestep
* **types** : *List[List[str]] (list of shape [timesteps, agents])*
    * A list containing timesteps, for each a list of the string name for the type of each agent
* **radii** : *np.ndarray (shape = [timesteps, agents])*
    * A numpy ndarray containing the radius for each agent at each timestep
    
Optionally, you can also include:
* **subpoints** : *np.ndarray (shape = [timesteps, agents, subpoints, 3])*
    * A numpy ndarray containing a list of subpoint position data for each agent at each timestep. These values are currently only used for fiber agents.
* **plots** : *Dict[str, Any]*
    * An object containing plot data already in Simularium format

### Generate example data

To demonstrate using the custom converter, we'll first generate some random example data.

In [2]:
from string import ascii_uppercase
from random import choice

# parameters
total_steps = 5
timestep = 0.5
box_size = 100
n_agents = 5
min_radius = 5
max_radius = 10
agents_are_fibers = False
points_per_fiber = 4

example_data = {
    "box_size" : np.array([box_size, box_size, box_size]),
    "times" : timestep * np.array(list(range(total_steps))),
    "n_agents" : np.array(total_steps * [n_agents]),
    "types" : []
}
for t in range(total_steps):
    example_data["types"].append([choice(ascii_uppercase) for i in range(n_agents)])

### Sphere visualization

If the agents are particles to be visualized as spheres, their centroid positions can be added like this:

In [3]:
if not agents_are_fibers:
    example_data["positions"] = np.random.uniform(size=(total_steps, n_agents, 3)) * box_size - box_size * 0.5
    example_data["viz_types"] = np.array(total_steps * [n_agents * [1000.0]]) # default viz type = 1000
    example_data["radii"] = (max_radius - min_radius) * np.random.uniform(size=(total_steps, n_agents)) + min_radius

### Line visualization

If the agents are fibers to be visualized as lines, the positions of points along each fiber can be added using the optional subpoints:

In [4]:
if agents_are_fibers:
    example_data["positions"] = np.zeros(shape=(total_steps, n_agents, 3))
    example_data["subpoints"] = box_size * np.random.uniform(
        size=(total_steps, n_agents, points_per_fiber, 3)) - box_size * 0.5
    example_data["viz_types"] = np.array(total_steps * [n_agents * [1001.0]]) # fiber viz type = 1001
    example_data["radii"] = np.zeros(shape=(total_steps, n_agents))

Inspect the resulting data:

In [5]:
print("box size = {}".format(example_data["box_size"]))
print("times = {}".format(example_data["times"]))
print("number of agents = {}".format(example_data["n_agents"]))
print("viz types = {}".format(example_data["viz_types"]))
print("positions = {}".format(example_data["positions"]))
print("types = {}".format(example_data["types"]))
print("radii = {}".format(example_data["radii"]))
if agents_are_fibers:
    print("subpoints = {}".format(example_data["subpoints"]))

box size = [100 100 100]
times = [0.  0.5 1.  1.5 2. ]
number of agents = [5 5 5 5 5]
viz types = [[1000. 1000. 1000. 1000. 1000.]
 [1000. 1000. 1000. 1000. 1000.]
 [1000. 1000. 1000. 1000. 1000.]
 [1000. 1000. 1000. 1000. 1000.]
 [1000. 1000. 1000. 1000. 1000.]]
positions = [[[  9.5312851   -9.2240184  -17.81748096]
  [ 27.93479996 -31.66498757   6.82480489]
  [ 34.25580236 -27.93179914 -49.07445191]
  [ 33.52012837  -8.44489426  28.55286451]
  [  1.5425217   37.98078987 -42.72384527]]

 [[-37.87303881 -44.67350242  40.26606988]
  [-31.30852262  -9.57517211  28.59633009]
  [ 46.07624816 -28.24140652 -33.97519619]
  [  8.36480235  -4.73641611  35.5778553 ]
  [-45.4707163   -0.39389959 -27.99590522]]

 [[-47.15608923   8.1161568   -2.67853815]
  [-49.19397705 -13.08796532  47.20505647]
  [-15.84389489  31.6293337   20.69530783]
  [ 38.33628897 -23.76067501  -1.08162202]
  [-25.25684242 -39.96311877 -43.9026804 ]]

 [[-27.1520531    5.83440243 -16.5381972 ]
  [ 32.78905856  32.06090865 -

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [6]:
converter = Converter(example_data)
converter.write_JSON("/Users/blairl/Desktop/example")

***
## Add plots

You can optionally include metrics data, which will be graphed in plots alongside your spatial data in the Simularium viewer. For now, Simularium supports scatterplots and histograms.

### Scatterplots

Scatterplots require the following data:
* **title** : *str*
    * A string display title for the plot
* **xaxis_title** : *str*
    * A string label (with units) for the x-axis
* **yaxis_title** : *str*
    * A string label (with units) for the y-axis
* **xtrace** : *np.ndarray (shape = [x values])*
    * A numpy ndarray of values for x, the independent variable
* **ytraces** : *Dict[str, np.ndarray (shape = [x values])]*
    * A dictionary with y-trace display names as keys, each mapped to a numpy ndarray of values for y, the dependent variable
* **render_mode** : *str (optional)*
    * A string specifying how to draw the datapoints. Options:
        * 'markers' : draw as points
        * 'lines' : connect points with line
    * Default: 'markers'

Here's some random example scatterplot data:

In [7]:
scatter_plot = {
    "title": "Test Scatterplot 1",
    "xaxis_title": "time (ns)",
    "yaxis_title": "concentration (uM)",
    "xtrace": np.array(list(range(10))),
    "ytraces": {
        "agent1": 100 * np.random.uniform(size=(10)),
        "agent2": 100 * np.random.uniform(size=(10)),
        "agent3": 100 * np.random.uniform(size=(10)),
    },
    "render_mode": "lines"
}

In [8]:
print("title = {}".format(scatter_plot["title"]))
print("xaxis_title = {}".format(scatter_plot["xaxis_title"]))
print("yaxis_title = {}".format(scatter_plot["yaxis_title"]))
print("xtrace = {}".format(scatter_plot["xtrace"]))
print("ytraces:")
print("  agent1 = {}".format(scatter_plot["ytraces"]["agent1"]))
print("  agent2 = {}".format(scatter_plot["ytraces"]["agent2"]))
print("  agent2 = {}".format(scatter_plot["ytraces"]["agent3"]))
print("render_mode = {}".format(scatter_plot["render_mode"]))

title = Test Scatterplot 1
xaxis_title = time (ns)
yaxis_title = concentration (uM)
xtrace = [0 1 2 3 4 5 6 7 8 9]
ytraces:
  agent1 = [19.35549636 60.25624246 95.32372489 36.61145118 42.58156768  6.34576689
  6.33212739 32.42412574 22.37868681 42.63289989]
  agent2 = [89.74526936 10.98069575  9.53999968 69.87143417 71.18053283 29.23632643
  0.87434113 20.30243689 26.48959454 23.42159043]
  agent2 = [57.09419399 76.57438212 63.78014238 15.79434719  6.066279   41.16115965
 68.14454336 55.71886738 46.43468864 28.87955534]
render_mode = lines


To add it to the converter:

In [9]:
converter.add_plot(scatter_plot, "scatter")

### Histograms

Histograms require the following data:
* **title** : *str*
    * A string display title for the plot
* ***xaxis_title*** : *str*
    * A string label (with units) for the x-axis
* **traces** : *Dict[str, np.ndarray (shape = [values])]*
    * A dictionary with trace display names as keys, each mapped to a numpy ndarray of values

Here's some random example histogram data:

In [10]:
histogram = {
    "title": "Test Histogram 1",
    "xaxis_title": "angle (degrees)",
    "traces": {
        "crosslinked monomers": 2 * np.random.uniform(size=(15)),
        "bent monomers": 10 * np.random.uniform(size=(10)),
    }
}

In [11]:
print("title = {}".format(histogram["title"]))
print("xaxis_title = {}".format(histogram["xaxis_title"]))
print("traces:")
print("  crosslinked monomers = {}".format(histogram["traces"]["crosslinked monomers"]))
print("  bent monomers = {}".format(histogram["traces"]["bent monomers"]))

title = Test Histogram 1
xaxis_title = angle (degrees)
traces:
  crosslinked monomers = [1.0169134  0.59357147 1.11322745 1.79753867 0.53335699 0.64159541
 1.23957633 0.961855   0.31864161 0.2253905  1.01900759 0.74037665
 0.67161706 0.65636688 1.54410297]
  bent monomers = [0.54369771 1.04864076 6.80810805 9.6930704  7.93284429 0.83999209
 5.9994699  4.12754526 7.20454793 4.49662127]


To add it to the converter:

In [12]:
converter.add_plot(histogram, "histogram")

### Save the data with added plots

Once you've added your plot data, write the updated data to file:

In [13]:
converter.write_JSON("/Users/blairl/Desktop/example")

## Visualize in the Simularium viewer

In your browser, either Firefox or Chrome, navigate to https://staging.agentviz.allencell.org/ and drag your file onto the center viewer window. 

*For now you'll first have to choose an example trajectory and close the load window. Once the example trajectory loads, you can drop your own file in to replace it. We'll fix this soon :)*